This notebook is used to explore PhoSim segmentation.txt file.  This file contains information on items like sensor position, read noise characteristics, x-talk values etc.  Using the functions in this notebook you can explore or extract the values yourself.  This is useful to (for example) see the x-talk values for a particular sensor presented in an easily digestible table format.

You can find this file in the PhoSim instruments directory.  So for example in:

phosim_release/data/lsst

You can drill down to the latest release version on the bitbucket site here:

https://bitbucket.org/phosim/phosim_release/src

In [1]:
import numpy as np
import pandas as pd

In [2]:
# For printing out matrix from 
# https://gist.github.com/braingineer/d801735dac07ff3ac4d746e1f218ab75

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end=" ")
        print("")

Read in the PhoSim segmentation.txt file

In [30]:
column_names = [
    'amp_name',
    'x_low', 'x_high', 'y_low', 'y_high',
    'serial_read',
    'parallel_read',
    'gain', 
    'gain_var',
    'bias_level', 
    'bias_var',
    'read_noise', 
    'read_noise_var',
    'dark_current',
    'dark_current_var',
    'parallel_prescan', 
    'serial_overscan', 
    'serial_prescan', 
    'parallel_overscan',
    'hot_pixel_rate',
    'hot_column_rate',
    'bits_per_pixel',
    'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 
    'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16'
]

segmentation = pd.read_csv('segmentation.txt', delim_whitespace=True, comment="#", header=None,
                           names=column_names)

In [31]:
# Look at first 5 entries
segmentation.head(5)

,amp_name,x_low,x_high,y_low,y_high,serial_read,parallel_read,gain,gain_var,bias_level,...,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16
0,R00_S12,16,4000,4072,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,R00_S12_C00,0,1999,0,508.00,1.00,-1.00,1.70,3.00,1000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,R00_S12_C01,0,1999,509,1017.00,1.00,-1.00,1.70,3.00,1000.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00
3,R00_S12_C02,0,1999,1018,1526.00,1.00,-1.00,1.70,3.00,1000.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
4,R00_S12_C03,0,1999,1527,2035.00,1.00,-1.00,1.70,3.00,1000.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00


Parse dataframe packing amplifiers together per sensor

In [32]:
full_list = []

row_generator = segmentation.iterrows()

for i,row in row_generator:
    
    # First read the sensor description
    sensor_name = row.amp_name     # Note: if a sensor and not an amp this is the sensor name
    number_amplifiers = row.x_low  # Note: if a sensor and not an amp this is the number of amplifiers
    
    # Now read each ammplifier line
    for m in range(number_amplifiers):
        j, amp = next(row_generator)

        new_list = [sensor_name] + amp.tolist()
        full_list.append(new_list)

In [33]:
new_dataframe = pd.DataFrame(full_list, columns=['Sensor'] + segmentation.columns.tolist())
new_dataframe.set_index(['Sensor','amp_name'], inplace=True)
grouped = new_dataframe.groupby('Sensor')
# new_dataframe[:16]

In [34]:
# Group rows together by sensor

R00_S21 = grouped.get_group('R00_S21')
R00_S12 = grouped.get_group('R00_S12')

R00_S22_C0 = grouped.get_group('R00_S22_C0')

In [35]:
pd.set_option('display.float_format', '{:.1g}'.format)
R00_S12.loc[:,'X1':'X16']

X1      X2     X3      X4      X5      X6     X7  \
Sensor  amp_name                                                            
R00_S12 R00_S12_C00       1  0.0006      0       0       0       0      0   
        R00_S12_C01  0.0007       1 0.0005  -9e-05  -8e-05  -9e-05 -3e-05   
        R00_S12_C02  -7e-05  0.0005      1  0.0005  -7e-05 -0.0001 -3e-05   
        R00_S12_C03 -0.0001  -9e-05 0.0004       1 -0.0002 -0.0001 -8e-05   
        R00_S12_C04 -0.0001 -0.0001 -9e-05 -0.0001       1  0.0004 -6e-05   
        R00_S12_C05  -1e-05  -2e-05 -2e-05  -2e-05  0.0009       1 0.0009   
        R00_S12_C06  -5e-05  -5e-05 -6e-05  -8e-05  -5e-05  0.0005      1   
        R00_S12_C07  -3e-05  -3e-05 -5e-05  -9e-05  -5e-05  -4e-05 0.0007   
        R00_S12_C10  -3e-05  -4e-05 -2e-05  -5e-05  -2e-05  -7e-05 -2e-05   
        R00_S12_C11  -2e-06  -6e-05 -6e-05  -1e-05  -4e-05  -7e-05 -3e-05   
        R00_S12_C12  -6e-05  -6e-05 -7e-05  -7e-05  -4e-06  -8e-05 -4e-05   
        R00_S12_C13  -6e-05  -7e-05 -5e-05  -7e-05  -4e-05  -2e-05 -3e-05   
        R00_S12_C14  -5e-05  -5e-05 -5e-05  -5e-05  -5e-05  -7e-05      0   
        R00_S12_C15  -5e-05  -6e-05 -6e-05  -7e-05  -6e-05  -7e-05 -3e-05   
        R00_S12_C16  -3e-05  -5e-05 -3e-05  -6e-05  -2e-05  -2e-05      0   
        R00_S12_C17  -2e-05  -3e-05 -3e-05  -5e-05  -3e-05  -4e-05      0   

                        X8     X9    X10    X11    X12     X13    X14    X15  \
Sensor  amp_name                                                               
R00_S12 R00_S12_C00      0      0      0      0      0       0      0      0   
        R00_S12_C01 -7e-06 -2e-05 -2e-05 -3e-05 -3e-05   8e-06 -4e-05 -3e-05   
        R00_S12_C02 -2e-05 -5e-06 -2e-05 -3e-05 -3e-05  -3e-05 -5e-05 -4e-05   
        R00_S12_C03 -3e-05 -3e-05 -1e-05 -5e-05 -6e-05  -4e-05 -6e-05 -6e-05   
        R00_S12_C04 -4e-05 -3e-05 -5e-05      0 -5e-05  -5e-05 -4e-05 -6e-05   
        R00_S12_C05      0      0 -2e-06 -4e-06      0  -6e-06 -8e-06 -2e-06   
        R00_S12_C06 0.0007 -2e-05 -2e-05 -3e-05 -4e-05       0 -3e-05 -4e-05   
        R00_S12_C07      1  3e-07 -9e-06 -1e-05 -2e-05  -2e-05      0 -3e-05   
        R00_S12_C10      0      1 0.0005 -3e-05 -3e-05  -4e-05 -4e-05 -5e-05   
        R00_S12_C11 -4e-06 0.0005      1 0.0002 -7e-05  -7e-05 -8e-05 -6e-05   
        R00_S12_C12 -3e-05 -5e-05 0.0003      1 0.0003  -8e-05 -7e-05 -7e-05   
        R00_S12_C13 -3e-05 -4e-05 -4e-05 0.0003      1 -0.0001 -7e-05 -6e-05   
        R00_S12_C14 -3e-05 -4e-05 -5e-05 -5e-05 -6e-05       1 0.0006 -7e-05   
        R00_S12_C15 -1e-05 -4e-05 -5e-05 -7e-05 -5e-05  0.0006      1 0.0002   
        R00_S12_C16      0      0 -1e-05 -2e-05      0  -9e-06 0.0004      1   
        R00_S12_C17      0      0 -1e-05 -3e-05 -5e-05  -2e-05 -2e-05 0.0005   

                       X16  
Sensor  amp_name            
R00_S12 R00_S12_C00      0  
        R00_S12_C01 -4e-05  
        R00_S12_C02 -4e-05  
        R00_S12_C03 -5e-05  
        R00_S12_C04 -6e-05  
        R00_S12_C05  9e-06  
        R00_S12_C06 -2e-05  
        R00_S12_C07 -3e-05  
        R00_S12_C10 -3e-05  
        R00_S12_C11 -6e-05  
        R00_S12_C12 -7e-05  
        R00_S12_C13 -5e-05  
        R00_S12_C14 -4e-05  
        R00_S12_C15 -5e-05  
        R00_S12_C16 0.0006  
        R00_S12_C17      1

In [36]:
# text_matrix = R00_S21.loc[:,'X1':'X16'].values
# matprint(text_matrix,".1e")

In [37]:
all_the_same = True

for name, group in grouped:
    
    # Skip wavefront sensors
    if len(group) == 8:
        continue
        
    matrix = group.loc[:,'X1':'X16'].values
    if not np.array_equal(matrix, R00_S21.loc[:,'X1':'X16'].values):
        print(name, len(group))
        all_the_same = False

print(all_the_same)

True


In [38]:
pd.set_option('display.float_format', '{:.2f}'.format)
R00_S12.loc[:,['read_noise', 'read_noise_var', 'dark_current','dark_current_var']]

read_noise  read_noise_var  dark_current  \
Sensor  amp_name                                                
R00_S12 R00_S12_C00        7.00            3.00          0.02   
        R00_S12_C01        7.00            3.00          0.02   
        R00_S12_C02        7.00            3.00          0.02   
        R00_S12_C03        7.00            3.00          0.02   
        R00_S12_C04        7.00            3.00          0.02   
        R00_S12_C05        7.00            3.00          0.02   
        R00_S12_C06        7.00            3.00          0.02   
        R00_S12_C07        7.00            3.00          0.02   
        R00_S12_C10        7.00            3.00          0.02   
        R00_S12_C11        7.00            3.00          0.02   
        R00_S12_C12        7.00            3.00          0.02   
        R00_S12_C13        7.00            3.00          0.02   
        R00_S12_C14        7.00            3.00          0.02   
        R00_S12_C15        7.00            3.00          0.02   
        R00_S12_C16        7.00            3.00          0.02   
        R00_S12_C17        7.00            3.00          0.02   

                     dark_current_var  
Sensor  amp_name                       
R00_S12 R00_S12_C00              0.00  
        R00_S12_C01              0.00  
        R00_S12_C02              0.00  
        R00_S12_C03              0.00  
        R00_S12_C04              0.00  
        R00_S12_C05              0.00  
        R00_S12_C06              0.00  
        R00_S12_C07              0.00  
        R00_S12_C10              0.00  
        R00_S12_C11              0.00  
        R00_S12_C12              0.00  
        R00_S12_C13              0.00  
        R00_S12_C14              0.00  
        R00_S12_C15              0.00  
        R00_S12_C16              0.00  
        R00_S12_C17              0.00

In [39]:
pd.set_option('display.float_format', '{:.8f}'.format)
R00_S12.loc[:,['hot_pixel_rate', 'hot_column_rate']]

hot_pixel_rate  hot_column_rate
Sensor  amp_name                                    
R00_S12 R00_S12_C00      0.00000012       0.00000012
        R00_S12_C01      0.00000012       0.00000012
        R00_S12_C02      0.00000012       0.00000012
        R00_S12_C03      0.00000012       0.00000012
        R00_S12_C04      0.00000012       0.00000012
        R00_S12_C05      0.00000012       0.00000012
        R00_S12_C06      0.00000012       0.00000012
        R00_S12_C07      0.00000012       0.00000012
        R00_S12_C10      0.00000012       0.00000012
        R00_S12_C11      0.00000012       0.00000012
        R00_S12_C12      0.00000012       0.00000012
        R00_S12_C13      0.00000012       0.00000012
        R00_S12_C14      0.00000012       0.00000012
        R00_S12_C15      0.00000012       0.00000012
        R00_S12_C16      0.00000012       0.00000012
        R00_S12_C17      0.00000012       0.00000012

In [40]:
pd.set_option('display.float_format', '{:.2f}'.format)
R00_S12.loc[:,['gain', 'bias_level', 'bias_var', 'read_noise', 'read_noise_var', 'dark_current','dark_current_var']]



gain  bias_level  bias_var  read_noise  read_noise_var  \
Sensor  amp_name                                                              
R00_S12 R00_S12_C00  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C01  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C02  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C03  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C04  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C05  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C06  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C07  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C10  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C11  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C12  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C13  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C14  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C15  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C16  1.70     1000.00      2.00        7.00            3.00   
        R00_S12_C17  1.70     1000.00      2.00        7.00            3.00   

                     dark_current  dark_current_var  
Sensor  amp_name                                     
R00_S12 R00_S12_C00          0.02              0.00  
        R00_S12_C01          0.02              0.00  
        R00_S12_C02          0.02              0.00  
        R00_S12_C03          0.02              0.00  
        R00_S12_C04          0.02              0.00  
        R00_S12_C05          0.02              0.00  
        R00_S12_C06          0.02              0.00  
        R00_S12_C07          0.02              0.00  
        R00_S12_C10          0.02              0.00  
        R00_S12_C11          0.02              0.00  
        R00_S12_C12          0.02              0.00  
        R00_S12_C13          0.02              0.00  
        R00_S12_C14          0.02              0.00  
        R00_S12_C15          0.02              0.00  
        R00_S12_C16          0.02              0.00  
        R00_S12_C17          0.02              0.00

In [41]:
R00_S12.loc[:,['bits_per_pixel']]

bits_per_pixel
Sensor  amp_name                   
R00_S12 R00_S12_C00           18.00
        R00_S12_C01           18.00
        R00_S12_C02           18.00
        R00_S12_C03           18.00
        R00_S12_C04           18.00
        R00_S12_C05           18.00
        R00_S12_C06           18.00
        R00_S12_C07           18.00
        R00_S12_C10           18.00
        R00_S12_C11           18.00
        R00_S12_C12           18.00
        R00_S12_C13           18.00
        R00_S12_C14           18.00
        R00_S12_C15           18.00
        R00_S12_C16           18.00
        R00_S12_C17           18.00